In [3]:
import numpy as np

def split_V(V):
    Vd = np.diag(np.diag(V))  # Diagonal part of V
    Vod = V - Vd  # Off-diagonal part of V
    return Vd, Vod
# Function to compute the Schrieffer-Wolff transformation (decomposed into H0, H1, H2)

def commutator(A,B):
    return A@B-B@A

# Function to compute the nested commutator [A, [A, ... [A, B] ...]]
def nested_commutator(A, B, n):
    result = B
    for _ in range(n):
        result = commutator(A, result)
    return result
def swt1(H0, V):
    dim = H0.shape[0]
    # Vd,Vod = split_V(V)
    # Energy differences between levels (diagonal of H0)
    delta = np.subtract.outer(np.diag(H0), np.diag(H0))  # Compute all pairwise energy gaps
    np.fill_diagonal(delta, 1)
    
    
    H1 = np.diag(np.diag(V))  # H1 is the diagonal part of V (first-order correction)
    # Initialize S and the Hamiltonian components
    S1 = V/delta
    
    H2 = 1/2*commutator(S1,V)
    S2 = commutator(S1,V)/delta/2
    np.fill_diagonal(S2, 0)
    
    H3 = 1/3*commutator(S1,commutator(S1,V))
    H4 = 1/8*nested_commutator(S1,V,3) + 1/4*commutator(S2,commutator(S1,V))

    return [S1,S2],[H1,H2,H3,H4]

def split_V(V):
    Vd = np.diag(np.diag(V))  # Diagonal part of V
    Vod = V - Vd  # Off-diagonal part of V
    return Vd, Vod
# Function to compute the Schrieffer-Wolff transformation (decomposed into H0, H1, H2)

def commutator(A,B):
    return A@B-B@A

def swt2(H0, V):
    dim = H0.shape[0]
    Vd,Vod = split_V(V)
    # Energy differences between levels (diagonal of H0)
    delta = np.subtract.outer(np.diag(H0), np.diag(H0))  # Compute all pairwise energy gaps
    np.fill_diagonal(delta, 1)
    
    
    H1 = np.diag(np.diag(V))  # H1 is the diagonal part of V (first-order correction)
    # Initialize S and the Hamiltonian components
    S1 = Vod/delta
    
    H2 = 1/2*commutator(S1,Vod)
    S2 = -commutator(Vd,S1)/delta

    H3 = 1/2*commutator(S2,Vod)
    S3 = (commutator(S2,Vd) + 1/3*nested_commutator(S1,Vod,2))/delta
    H4 = 1/2*commutator(S3,Vod) - 1/24 *nested_commutator(S1,Vod,3)

    
    return [S1,S2,S3],[H1,H2,H3,H4]


In [8]:
def split_V(V):
    Vd = np.diag(np.diag(V))  # Diagonal part of V
    Vod = V - Vd  # Off-diagonal part of V
    return Vd, Vod

def commutator(A, B):
    return A @ B - B @ A

def nested_commutator(A, B, n):
    if n == 1:
        return commutator(A, B)
    else:
        return commutator(A, nested_commutator(A, B, n-1))
def create_subspace_projector(dim, subspace_indices):
    """
    Create a projector for the given subspace.

    Args:
        dim (int): The total dimension of the Hilbert space.
        subspace_indices (list or np.ndarray): The indices of the subspace.

    Returns:
        np.ndarray: The projector for the given subspace.
    """
    # Create the basis states for the subspace
    basis_states = [np.zeros(dim) for _ in range(len(subspace_indices))]
    for i, idx in enumerate(subspace_indices):
        basis_states[i][idx] = 1

    # Compute the projector
    projector = np.array([state[:, None] @ state[None, :] for state in basis_states])
    P = np.sum(projector, axis=0)
    Q = np.identity(dim) - P
    return P, Q
def swt_subspace(H0, V, subspace_indices):
    """
    Compute the Schrieffer-Wolff transformation on a subspace of the Hamiltonian.

    Args:
        H0 (np.ndarray): The unperturbed Hamiltonian.
        V (np.ndarray): The perturbation Hamiltonian.
        subspace_indices (list or np.ndarray): The indices of the subspace to consider.

    Returns:
        S (list): The Schrieffer-Wolff transformation operators (S1, S2, S3).
        H (list): The transformed Hamiltonian components (H1, H2, H3, H4).
    """
    dim = H0.shape[0]
    subspace_dim = len(subspace_indices)
    P,Q = create_subspace_projector(dim, subspace_indices)
    
    # Extract the subspace of H0 and V
    
    Vd = P@V@P + Q@V@Q
    Vod = P@V@Q + Q@V@P

    # Compute the energy differences in the subspace
    delta = np.subtract.outer(np.diag(H0), np.diag(H0))
    np.fill_diagonal(delta, 1)

    # Compute the Schrieffer-Wolff transformation components
    H1 = Vd
    S1= Vod/ delta

    H2 = 1/2 * commutator(S1, Vod)
    S2 = -commutator(Vd,S1)/delta

    H3 = 1/2*commutator(S2,Vod)
    S3 = (commutator(S2,Vd) + 1/3*nested_commutator(S1,Vod,2))/delta
    H4 = 1/2*commutator(S3,Vod) - 1/24 *nested_commutator(S1,Vod,3)

    return [S1,S2,S3],[H1,H2,H3,H4]

In [10]:
import numpy as np
np.set_printoptions(edgeitems=30, linewidth=100000, 
    formatter=dict(float=lambda x: "%.3g" % x))

In [14]:
# Define the unperturbed Hamiltonian (H0)
H0 = np.array([[1.0, 0.1, 0.2, 0.3, 0.1, 0.2, 0.3, 0.4],
               [0.1, 2.0, 0.4, 0.5, 0.2, 0.3, 0.4, 0.5],
               [0.2, 0.4, 3.0, 0.6, 0.3, 0.4, 0.5, 0.6],
               [0.3, 0.5, 0.6, 4.0, 0.4, 0.5, 0.6, 0.7],
               [0.1, 0.2, 0.3, 0.4, 5.0, 0.6, 0.7, 0.8],
               [0.2, 0.3, 0.4, 0.5, 0.6, 6.0, 0.8, 0.9],
               [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 7.0, 1.0],
               [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 8.0]])

# Define the perturbation Hamiltonian (V)
V = np.array([[0.0, 0.01, 0.02, 0.03, 0.01, 0.02, 0.03, 0.04],
              [0.01, 0.0, 0.04, 0.05, 0.02, 0.03, 0.04, 0.05],
              [0.02, 0.04, 0.0, 0.06, 0.03, 0.04, 0.05, 0.06],
              [0.03, 0.05, 0.06, 0.0, 0.04, 0.05, 0.06, 0.07],
              [0.01, 0.02, 0.03, 0.04, 0.0, 0.06, 0.07, 0.08],
              [0.02, 0.03, 0.04, 0.05, 0.06, 0.0, 0.08, 0.09],
              [0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.0, 0.10],
              [0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.10, 0.0]])

# Define the subspace indices
subspace_indices = list(range(4))  # Subspace from 0 to 3

# Compute the Schrieffer-Wolff transformation for the subspace
S, H = swt_subspace(H0, V, subspace_indices)

# Print the Schrieffer-Wolff transformation components
print("Schrieffer-Wolff Transformation Components:")
print("S1:")
print(S[0])


print("\nTransformed Hamiltonian Components:")
print("H1:")
print(H[0])
print("H2:")
print(H[1])
print("H3:")
print(H[2])
print("H4:")
print(H[3])


Schrieffer-Wolff Transformation Components:
S1:
[[0 -0 -0 -0 -0.0025 -0.004 -0.005 -0.00571]
 [0 0 -0 -0 -0.00667 -0.0075 -0.008 -0.00833]
 [0 0 0 -0 -0.015 -0.0133 -0.0125 -0.012]
 [0 0 0 0 -0.04 -0.025 -0.02 -0.0175]
 [0.0025 0.00667 0.015 0.04 0 -0 -0 -0]
 [0.004 0.0075 0.0133 0.025 0 0 -0 -0]
 [0.005 0.008 0.0125 0.02 0 0 0 -0]
 [0.00571 0.00833 0.012 0.0175 0 0 0 0]]

Transformed Hamiltonian Components:
H1:
[[0 0.01 0.02 0.03 0 0 0 0]
 [0.01 0 0.04 0.05 0 0 0 0]
 [0.02 0.04 0 0.06 0 0 0 0]
 [0.03 0.05 0.06 0 0 0 0 0]
 [0 0 0 0 0 0.06 0.07 0.08]
 [0 0 0 0 0.06 0 0.08 0.09]
 [0 0 0 0 0.07 0.08 0 0.1]
 [0 0 0 0 0.08 0.09 0.1 0]]
H2:
[[-0.000484 -0.000723 -0.00105 -0.0016 0 0 0 0]
 [-0.000723 -0.0011 -0.0016 -0.00247 0 0 0 0]
 [-0.00105 -0.0016 -0.00233 -0.00355 0 0 0 0]
 [-0.0016 -0.00247 -0.00355 -0.00528 0 0 0 0]
 [0 0 0 0 0.00221 0.00222 0.00244 0.00271]
 [0 0 0 0 0.00222 0.00209 0.00221 0.0024]
 [0 0 0 0 0.00244 0.00221 0.0023 0.00245]
 [0 0 0 0 0.00271 0.0024 0.00245 0.00259]]
H